# QRT : Stock return prediction

## Loading libraries

In [1]:
#import librairies

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

#Preprocessing, encoding etc..


from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

#Algorithms

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier

#pd.set_option('display.max_row',100)
pd.set_option('display.max_column',47)

print("Setup complete ! ")

Setup complete ! 


## Loading data

The train and test inputs are composed of 46 features.

The target of this challenge is `RET` and corresponds to the fact that the **return is in the top 50% of highest stock returns**.

Since the median is very close to 0, this information should not change much with the idea to predict the sign of the return.

### Train Set

In [2]:
X_train = pd.read_csv('../Data/x_train.csv', index_col='ID')
y_train = pd.read_csv('../Data/y_train.csv', index_col='ID')
df_train = pd.concat([X_train, y_train], axis=1) # ou pd.merge(right= y_train, left=x_train, on='ID')
df_train.head()

,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,VOLUME_2,RET_3,VOLUME_3,RET_4,VOLUME_4,RET_5,VOLUME_5,RET_6,VOLUME_6,RET_7,VOLUME_7,RET_8,VOLUME_8,RET_9,VOLUME_9,RET_10,VOLUME_10,RET_11,VOLUME_11,RET_12,VOLUME_12,RET_13,VOLUME_13,RET_14,VOLUME_14,RET_15,VOLUME_15,RET_16,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20,RET
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,2,18,5,3,44,-0.015748,0.147931,-0.015504,0.179183,0.010972,0.033832,-0.014672,-0.362868,0.016483,-0.972920,0.014331,0.590810,-0.017215,0.557246,-0.018433,0.453590,0.134146,8.580442,0.182287,3.427303,0.013570,-0.328418,0.030108,-0.816436,0.027624,-0.244636,-0.049370,-2.042624,-0.028571,-0.369605,0.059459,0.630899,0.003254,-0.379412,0.008752,-0.110597,-0.012959,0.174521,-0.002155,-0.000937,True
1,0,3,43,15,6,104,0.003984,NaN,-0.090580,NaN,0.018826,NaN,-0.025540,NaN,-0.038062,NaN,-0.006873,NaN,-0.026756,NaN,0.023973,NaN,0.158731,NaN,0.045642,NaN,-0.008230,NaN,-0.028000,NaN,-0.019608,NaN,-0.052044,NaN,-0.004073,NaN,0.015413,NaN,0.003774,NaN,-0.018518,NaN,-0.028777,NaN,-0.034722,NaN,True
2,0,4,57,20,8,142,0.000440,-0.096282,-0.058896,0.084771,-0.009042,-0.298777,0.024852,-0.157421,0.009354,0.091455,0.027522,-0.405978,-0.023047,-0.389777,-0.002979,-0.701838,-0.006342,-0.229531,0.055803,0.543460,-0.012781,-0.121126,0.028093,-0.187444,-0.009426,-0.081783,-0.002686,-0.356711,-0.007552,-0.336773,0.008964,-0.010336,-0.017612,-0.354333,-0.006562,-0.519391,-0.012101,-0.356157,-0.006867,-0.308868,False
3,0,8,1,1,1,2,0.031298,-0.429540,0.007756,-0.089919,-0.004632,-0.639737,-0.019677,-0.940163,0.003544,-0.882464,-0.002021,-0.113951,-0.043962,-0.395345,-0.004329,-0.765754,0.014146,-0.593888,-0.021946,0.588171,0.007693,-0.427358,0.007264,-0.540519,-0.010067,-0.839232,0.000479,-2.144763,0.021057,-0.431029,-0.031769,0.012105,0.033824,-0.290178,-0.001468,-0.663834,-0.013520,-0.562126,-0.036745,-0.631458,False
4,0,14,36,12,5,92,0.027273,-0.847155,-0.039302,-0.943033,0.000000,-1.180629,0.000000,-1.313896,0.022321,-1.204398,0.018182,-0.879172,-0.026549,-0.858613,-0.017391,-0.029690,-0.008621,-1.022991,0.022026,-0.387114,-0.008734,-0.288180,-0.004348,-0.115091,-0.041667,0.271702,0.095891,18.894266,-0.026667,0.803608,-0.038461,-0.277083,-0.012659,0.139086,0.004237,-0.017547,0.004256,0.579510,-0.040817,0.802806,False


In [3]:
print(f'Number of rows in train set : {df_train.shape[0]}')
print(f'Number of columns in train set : {df_train.shape[1]}')

Number of rows in train set : 418595
Number of columns in train set : 47


### Test Set

In [4]:
df_test = pd.read_csv('../Data/x_test.csv', index_col='ID')
df_test.head()

,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,VOLUME_2,RET_3,VOLUME_3,RET_4,VOLUME_4,RET_5,VOLUME_5,RET_6,VOLUME_6,RET_7,VOLUME_7,RET_8,VOLUME_8,RET_9,VOLUME_9,RET_10,VOLUME_10,RET_11,VOLUME_11,RET_12,VOLUME_12,RET_13,VOLUME_13,RET_14,VOLUME_14,RET_15,VOLUME_15,RET_16,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
418595,2,0,37,12,5,94,0.020208,0.146176,0.010059,0.224756,0.002192,-0.474404,0.001903,0.026316,0.001466,0.079495,0.000586,-0.078154,-0.001611,-0.243718,-0.000146,-0.162995,-0.004519,-0.082635,0.003071,-0.261942,-0.013843,-0.385339,-0.002876,-0.589507,0.003463,0.060668,-0.001872,-0.249247,0.005939,-0.418752,0.022364,-0.301163,-0.001035,-0.416533,-0.000148,-0.004548,-0.000148,-0.161792,0.016997,-0.007221
418596,2,1,15,4,3,37,0.009134,-0.251631,0.021913,-0.712515,-0.007251,-0.328719,-0.017223,-0.921532,-0.019126,-0.857393,0.010695,-0.982053,-0.017356,-0.493607,0.008460,-0.797030,0.015403,-0.084381,0.016867,0.269908,-0.022475,-0.295082,0.040968,1.766451,0.006478,-0.386691,-0.012690,-0.525177,0.017562,-0.530799,-0.002062,-0.679724,-0.001544,-0.408979,0.001546,0.396372,-0.007875,-0.431760,0.001742,-0.574228
418597,2,4,57,20,8,142,0.005008,-0.115845,0.005914,-0.107441,-0.003927,0.347900,0.000873,0.731513,-0.086193,2.681007,0.028314,0.428653,-0.012561,0.323244,0.018152,0.037231,-0.005335,0.219277,0.000000,0.032683,-0.004494,-0.099527,0.002458,0.263624,0.000000,-0.336112,-0.006913,-0.039828,-0.007668,0.377134,0.004458,-0.451590,0.011481,-0.536967,0.009520,-0.368585,0.000000,0.022713,-0.002066,-0.207362
418598,2,5,35,12,5,91,0.015370,-0.090295,-0.013738,0.048465,0.008600,-0.218245,0.001915,-0.055296,0.008691,0.257485,0.016442,0.901757,0.027742,0.863997,0.006857,-0.930870,0.002546,-0.183697,0.000000,-0.399050,0.001785,-0.790012,-0.012840,-0.518844,0.008377,-0.350864,0.004847,-0.250060,-0.010600,-0.576994,-0.010984,-0.039714,0.037018,0.665132,-0.003097,0.141991,-0.008191,-0.172382,0.005145,-0.353172
418599,2,6,57,20,8,142,0.011419,-0.289027,0.022807,-0.262690,0.014645,-0.370676,-0.047796,-0.107222,-0.010220,-0.412849,0.024504,-0.289160,-0.004834,-0.658439,-0.004065,-0.847188,0.005270,-0.361449,0.014622,-0.739579,-0.001851,-0.785226,0.003608,-0.503767,-0.019612,-0.285158,-0.022215,-0.368104,-0.003550,-0.729463,0.001045,-0.565747,0.004304,-0.506291,-0.026469,-0.280666,0.010743,0.365773,-0.011134,0.933284


In [5]:
print(f'Number of rows in test set : {df_test.shape[0]}')
print(f'Number of columns in test set : {df_test.shape[1]}')

Number of rows in test set : 198429
Number of columns in test set : 46


Input datasets comprise 47 columns: the first ID column contains unique row identifiers while the other 46 descriptive features correspond to:

- `DATE`: an index of the date (the dates are randomized and anonymized so there is no continuity or link between any dates),
- `STOCK`: an index of the stock,
- `INDUSTRY`: an index of the stock industry domain (e.g., aeronautic, IT, oil company),
- `INDUSTRY_GROUP`: an index of the group industry,
- `SUB_INDUSTRY`: a lower level index of the industry,
- `SECTOR`: an index of the work sector,
- `RET_1` to `RET_20`: the historical residual returns among the last 20 days (i.e., RET_1 is the return of the previous day and so on),
- `VOLUME_1` to `VOLUME_20`: the historical relative volume traded among the last 20 days (i.e., VOLUME_1 is the relative volume of the previous day and so on),

Output datasets are only composed of 2 columns:

- `ID`: the unique row identifier (corresponding to the input identifiers)
and the binary target:
- `RET`: the sign of the residual stock return at time t

## Exploratory Data Analysis

In [6]:
y_train.value_counts()

RET  
False    209749
True     208846
dtype: int64

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418595 entries, 0 to 418594
Data columns (total 47 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            418595 non-null  int64  
 1   STOCK           418595 non-null  int64  
 2   INDUSTRY        418595 non-null  int64  
 3   INDUSTRY_GROUP  418595 non-null  int64  
 4   SECTOR          418595 non-null  int64  
 5   SUB_INDUSTRY    418595 non-null  int64  
 6   RET_1           416236 non-null  float64
 7   VOLUME_1        353570 non-null  float64
 8   RET_2           416130 non-null  float64
 9   VOLUME_2        352209 non-null  float64
 10  RET_3           416088 non-null  float64
 11  VOLUME_3        350776 non-null  float64
 12  RET_4           416051 non-null  float64
 13  VOLUME_4        347598 non-null  float64
 14  RET_5           416011 non-null  float64
 15  VOLUME_5        343902 non-null  float64
 16  RET_6           415998 non-null  float64
 17  VOLUME_6  

In [8]:
df_train.isna().sum()

DATE                  0
STOCK                 0
INDUSTRY              0
INDUSTRY_GROUP        0
SECTOR                0
SUB_INDUSTRY          0
RET_1              2359
VOLUME_1          65025
RET_2              2465
VOLUME_2          66386
RET_3              2507
VOLUME_3          67819
RET_4              2544
VOLUME_4          70997
RET_5              2584
VOLUME_5          74693
RET_6              2597
VOLUME_6          74714
RET_7              2585
VOLUME_7          73853
RET_8              2623
VOLUME_8          73898
RET_9              2682
VOLUME_9          73298
RET_10             2692
VOLUME_10         73305
RET_11             2961
VOLUME_11         72025
RET_12             3186
VOLUME_12         62523
RET_13             3360
VOLUME_13         59008
RET_14             4413
VOLUME_14         60929
RET_15             4990
VOLUME_15         66373
RET_16             5280
VOLUME_16         67262
RET_17             5301
VOLUME_17         62314
RET_18             5307
VOLUME_18       

In [9]:
df_test.isna().sum()

DATE                  0
STOCK                 0
INDUSTRY              0
INDUSTRY_GROUP        0
SECTOR                0
SUB_INDUSTRY          0
RET_1                 8
VOLUME_1          27190
RET_2                17
VOLUME_2          25394
RET_3                22
VOLUME_3          27371
RET_4                33
VOLUME_4          29199
RET_5                42
VOLUME_5          31095
RET_6                45
VOLUME_6          31102
RET_7                46
VOLUME_7          31109
RET_8                44
VOLUME_8          31120
RET_9                40
VOLUME_9          31128
RET_10               60
VOLUME_10         29253
RET_11             1053
VOLUME_11         28285
RET_12              166
VOLUME_12         25360
RET_13              274
VOLUME_13         25486
RET_14             1662
VOLUME_14         28613
RET_15             2019
VOLUME_15         28934
RET_16             1325
VOLUME_16         26351
RET_17             1398
VOLUME_17         28173
RET_18             1582
VOLUME_18       

- There is a lot of missing values in both set and we can't drop them because we'll lost too much informations

In [10]:
df_train['STOCK'].nunique()

5350

In [11]:
df_train['SECTOR'].nunique()

12

In [12]:
df_train['INDUSTRY'].nunique()

72

In [13]:
df_train['INDUSTRY_GROUP'].nunique()

26

## Feature Engineering

To create new features and selects some of them, we create classes from BaseEstimator and TrabsformerMixin.

By default, the class Features_engi creates feature that describe the median of `RET_1` , `RET_5`, `RET_11`, `RET_15`, `RET_20` conditionally to the `SECTOR` and the `DATE` and that describe also the median of `VOLUME_1`,`VOLUME_5`, `VOLUME_11`, `VOLUME_15`, `VOLUME_20` conditionally to the `STOCK` and `INDUSTRY`.

In [14]:
class Features_engi(BaseEstimator, TransformerMixin):
    
    def __init__(self, shifts = [1, 5, 11, 15, 20], statistics = ['median'], target_feature = ['RET'], gb_features = ['SECTOR','DATE','STOCK']):
        self.shifts = shifts # To deal with noise we can't take all features
        self.statistics = statistics # type of statistic 
        self.target_feature = target_feature 
        self.gb_features = gb_features
        self.tmp_name = str()
        self.new_features = []
        
        
    def fit(self, X, y=None):  
        self.tmp_name = '_'.join(self.gb_features)
        return self
        
    def transform(self, X, y=None):
        #X_ = pd.DataFrame(X).copy()
        for shift in self.shifts:
            for stat in self.statistics:
                for target in self.target_feature :
                    name = f'{target}_{shift}_{self.tmp_name}_{stat}'
                    feat = f'{target}_{shift}'
                    self.new_features.append(name)
                    X[name] = X.groupby(self.gb_features)[feat].transform(stat)
        features = list(X.columns)
        features += self.new_features  # We add conditional features
        X_ = X[features].copy()
        X_ = X_.loc[:,~X_.columns.duplicated()] 
        X_[X_==np.inf]=np.nan #To handle with infinite values
        return X_

The class Features_select select features to keep.
We have created a list of features pre-defined, with all new features, `RET_1` to `RET_20` and `VOLUME_1` to `VOLUME_20`

In [15]:
class Features_select(BaseEstimator, TransformerMixin):
    
    def __init__(self, select = None) :
        self.select = select
        self.features_select = list()
              
    def fit(self, X, y=None): 
        if self.select == None :
            self.features_select = list(X.columns.copy()) # If we want to keep all feature
            
        elif self.select == 'pre-defini' : # Pre defined list of features
            self.features_select += ['RET_%d' % (i+1) for i in range(20)]
            self.features_select += ['VOLUME_%d' % (i+1) for i in range(20)]
            self.features_select += ['RET_%d_SECTOR_DATE_STOCK_median' % (i) for i in [1, 5, 11, 15, 20]]
            self.features_select += ['VOLUME_%d_STOCK_INDUSTRY_median' % (i) for i in [1, 5, 11, 15, 20]]

        else : 
            self.features_select = self.select.copy() #
        return self
        
    def transform(self, X, y=None):
        X_ = X[self.features_select].copy()
        X_ = X_.loc[:,~X_.columns.duplicated()] 
        return X_

## Modelling

In [16]:
y_train = y_train.RET

### Evaluation

- To evaluate our model we'll be using a 5-stratified-fold cross validation with the accuracy since it's the metric that the competition uses in the leaderboard.

- To do that, we'll define a small scoring function.

In [17]:
st_cv = StratifiedKFold(5)

In [18]:
def cv_score(clf, X, y, scoring='accuracy'):
    xval = cross_val_score(clf, X, y, cv = st_cv, scoring=scoring)
    print(f'Accuracy : {np.mean(xval)}')

### Bagging vs Boosting

We have chosen Boosting and Bagging methods

We create 8 different models : 

- Default variables creations, pre-defined features selections, missing values imputed by 0, Random Forest

- Default variables creations, pre-defined features selections, missing values imputed by the mean, Random Forest

- Default variables creations, pre-defined features selections, missing values imputed by the median, Random Forest

- Variables creations with an other method, features selections by a list, missing values imputed by the mean, Random Forest



- Default variables creations, pre-defined features selections, missing values imputed by 0, Xgboost

- Default variables creations, pre-defined features selections, missing values imputed by the mean, Xgboost

- Default variables creations, pre-defined features selections, missing values imputed by the median, Xgboost

- Variables creations with an other method, features selections by a list, missing values imputed by the mean, Xgboost


#### Bagging

In [19]:
clf_zero_rf = Pipeline([("engin1",Features_engi()), 
                        ('engin2', Features_engi(target_feature = ['VOLUME'], gb_features = ['STOCK','INDUSTRY'])),
                        ('selection', Features_select('pre-defini')),
                        ("zero", SimpleImputer(missing_values=np.nan,strategy='constant', fill_value=0)), 
                        ("rf", RandomForestClassifier(n_estimators= 200, max_depth= 5,random_state=0))])

In [20]:
cv_score(clf_zero_rf, X_train, y_train)

Accuracy : 0.5151614328885916


In [21]:
clf_mean_rf = Pipeline([("engin1", Features_engi()), 
                        ('engin2', Features_engi(target_feature = ['VOLUME'], gb_features = ['STOCK','INDUSTRY'])),
                        ('selection', Features_select('pre-defini')),
                        ("mean", SimpleImputer(missing_values=np.nan,strategy='mean')),
                        ("rf", RandomForestClassifier(n_estimators= 200, max_depth= 5,random_state=0))])

In [22]:
cv_score(clf_mean_rf, X_train, y_train)

Accuracy : 0.5151948781041341


In [23]:
clf_median_rf = Pipeline([("engin1", Features_engi()), 
                        ('engin2', Features_engi(target_feature = ['VOLUME'], gb_features = ['STOCK','INDUSTRY'])),
                        ('selection', Features_select('pre-defini')),
                        ("mean", SimpleImputer(missing_values=np.nan,strategy='median')), 
                        ("rf", RandomForestClassifier(n_estimators= 200, max_depth= 5,random_state=0))])

In [24]:
cv_score(clf_median_rf, X_train, y_train)

Accuracy : 0.5147433676943107


In [25]:
lst = ['RET_%d' % (i+1) for i in range(5)]
lst += ['VOLUME_%d' % (i+1) for i in range(5)]
lst += ['VOLUME_%d_SECTOR_INDUSTRY_STOCK_median' % (i+1) for i in range(5)]
lst += ['RET_%d_SECTOR_INDUSTRY_STOCK_median' % (i+1) for i in range(5)]

            
clf_creation_rf = Pipeline([("engin1", Features_engi(shifts = [1,2,3,4,5], statistics = ['median'], 
                                                     target_feature = ['RET', 'VOLUME'],
                                                     gb_features = ['SECTOR','INDUSTRY','STOCK'])), 
                        ('selection', Features_select(lst)),
                        ("mean", SimpleImputer(missing_values=np.nan,strategy='mean')),
                        ("rf", RandomForestClassifier(n_estimators= 200, max_depth= 5,random_state=0))])

In [26]:
cv_score(clf_creation_rf, X_train, y_train)

Accuracy : 0.5119005243732009


#### Boosting

In [27]:
clf_zero_xgb = Pipeline([("engin1",Features_engi()), 
                        ('engin2', Features_engi(target_feature = ['VOLUME'], gb_features = ['STOCK','INDUSTRY'])),
                        ('selection', Features_select('pre-defini')),
                        ("zero", SimpleImputer(missing_values=np.nan,strategy='constant', fill_value=0)), 
                        ("xgb", XGBClassifier(eval_metric = 'logloss',random_state=0))])

In [28]:
cv_score(clf_zero_xgb, X_train, y_train)

Accuracy : 0.5079969899306012


In [29]:
clf_mean_xgb = Pipeline([("engin1", Features_engi()), 
                        ('engin2', Features_engi(target_feature = ['VOLUME'], gb_features = ['STOCK','INDUSTRY'])),
                        ('selection', Features_select('pre-defini')),
                        ("mean", SimpleImputer(missing_values=np.nan,strategy='mean')),
                        ("xgb", XGBClassifier(eval_metric = 'logloss',random_state=0))])

In [30]:
cv_score(clf_mean_xgb, X_train, y_train)

Accuracy : 0.506790573227105


In [31]:
clf_median_xgb = Pipeline([("engin1", Features_engi()), 
                        ('engin2', Features_engi(target_feature = ['VOLUME'], gb_features = ['STOCK','INDUSTRY'])),
                        ('selection', Features_select('pre-defini')),
                        ("mean", SimpleImputer(missing_values=np.nan,strategy='median')), 
                        ("xgb", XGBClassifier(eval_metric = 'logloss',random_state=0))])

In [32]:
cv_score(clf_median_xgb, X_train, y_train)

Accuracy : 0.5060404448213667


In [33]:
lst = ['RET_%d' % (i+1) for i in range(5)]
lst += ['VOLUME_%d' % (i+1) for i in range(5)]
lst += ['VOLUME_%d_SECTOR_INDUSTRY_STOCK_median' % (i+1) for i in range(5)]
lst += ['RET_%d_SECTOR_INDUSTRY_STOCK_median' % (i+1) for i in range(5)]

            
clf_creation_xgb = Pipeline([("engin1", Features_engi(shifts = [1,2,3,4,5], statistics = ['median'],
                                                      target_feature = ['RET', 'VOLUME'],
                                                      gb_features = ['SECTOR','INDUSTRY','STOCK'])), 
                        ('selection', Features_select(lst)),
                        ("mean", SimpleImputer(missing_values=np.nan,strategy='mean')),
                        ("xgb", XGBClassifier(eval_metric = 'logloss',random_state=0))])

In [34]:
cv_score(clf_creation_xgb, X_train, y_train)

Accuracy : 0.50536915156655


- We select Bagging method and try to improve the score of the `clf_mean_rf`

### Tuning parameters

In [35]:
def Grid_Search(model):
    
    grid = GridSearchCV(model, params, scoring='accuracy', cv=st_cv)
                       
    grid.fit(X_train,y_train)
    
    print("Best parameters by grid search:\n", grid.best_params_)
    print(f'Accuracy {grid.score(X_train, y_train)}')
    
    return grid

In [38]:
#It takes too much time to try all theses parameters so we won't tune our RandomForest for now
#params = {'rf__n_estimators' : [500,600],
          #'rf__max_depth' : [5,10,15],
          #'rf__min_samples_split': [1, 2, 3],
          #'min_samples_leaf': [1, 3, 10],
         #}

In [41]:
#rf_tuned = Grid_Search(clf_mean_rf)

### Final model

In [40]:
final_model =  clf_mean_rf #rf_tuned.best_estimator_

final_model.fit(X_train, y_train)
print(f'Accuracy on train set :{np.round(final_model.score(X_train, y_train),3)}')

Accuracy on train set :0.531


## Generate the subsmission

In [43]:
X_test = df_test

In [70]:
y_pred = final_model.predict_proba(X_test)[:,1]

In [72]:
sub = df_test.copy()
sub['pred'] = y_predproba
y_pred = sub.groupby('DATE')['pred'].transform(
    lambda x: x > x.median()).values

In [75]:
submission = pd.Series(y_pred)
submission.index = X_test.index
submission.to_csv('../Data/submissions_qrt.csv', index=True, header=True)

In [76]:
#Other possiblity
#y_pred = final_model.predict(X_test)
#submission = pd.Series(y_pred)
#submission.index = X_test.index
#submission.to_csv('../Data/submissions_qrt.csv', index=True, header=True)